In [1]:
import os
from openai import OpenAI
from pydantic import BaseModel

from langchain.output_parsers import PydanticOutputParser

In [2]:
# Check if the API key is set
assert 'OPENAI_API_KEY' in os.environ, 'Please set the OPENAI_API_KEY environment variable'

In [3]:
LLM = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [4]:
# Create a model to define the structure of the data you want to extract.
# This model will map directly to the information in the prompt.
class UserDetail(BaseModel):
    """User detail data model."""
    name: str
    age: int

parser = PydanticOutputParser(pydantic_object=UserDetail)

In [5]:
query_text = "Name: John Doe\nAge: 30"

In [13]:
prompt_template = """
Extract the data following the format:
{format_instructions}

Query text:
{query_text}
"""

prompt = prompt_template.format(
    format_instructions=parser.get_format_instructions(),
    query_text=query_text
)

In [15]:
chat_completion = LLM.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
)

In [16]:
# extract the response
response = chat_completion.choices[0].message.content
print(response)

{
  "name": "John Doe",
  "age": 30
}


In [17]:
user: UserDetail = parser.parse(response)
print(user)

name='John Doe' age=30
